In [1]:
import os 
api_key = os.environ.get("OPENAI_API_KEY")

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests
from pydantic import BaseModel

In [5]:
## Tool Create
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency:str,target_currency:str) -> float:
    '''
    This function fetches the currency conversion factor between a given base currency and a target currency
    '''
    url = f"https://v6.exchangerate-api.com/v6/ebf705fd2bdb667f968e817b/pair/{base_currency}/{target_currency}"
    response = requests.get(url)    
    return response.json()

@tool 
def convert(base_currency_value:int,conversion_rate:Annotated[float,InjectedToolArg]) -> float:
    """
    Given a currency conversion rate this function calculates hte target currency value from a given base currency value 
    """
    
    return base_currency_value * conversion_rate

In [7]:
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'}}

In [8]:

get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1769385601,
 'time_last_update_utc': 'Mon, 26 Jan 2026 00:00:01 +0000',
 'time_next_update_unix': 1769472001,
 'time_next_update_utc': 'Tue, 27 Jan 2026 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 91.8161}

In [10]:
convert.invoke({"base_currency_value":10,'conversion_rate':89.16})

891.5999999999999

In [11]:
## Tool Binding
llm = ChatOpenAI()

In [12]:
llm_with_tools = llm.bind_tools([get_conversion_factor,convert])

In [13]:
# tool calling
messages = [HumanMessage('What is the conversion factor between USD and INR, and based on that can you convert 10 usd to inr')]

In [14]:
messages

[HumanMessage(content='What is the conversion factor between USD and INR, and based on that can you convert 10 usd to inr', additional_kwargs={}, response_metadata={})]

In [15]:
ai_message = llm_with_tools.invoke(messages)

In [16]:
messages.append(ai_message)

In [17]:
ai_message

AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 124, 'total_tokens': 176, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-D2IeIYNjDngshaT34T0dNWGjDuyA5', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bfae2-9aa4-7be3-a5bf-f9a445c2332d-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'base_currency': 'USD', 'target_currency': 'INR'}, 'id': 'call_obxBud7gMEUPJGeSrcLJ0qov', 'type': 'tool_call'}, {'name': 'convert', 'args': {'base_currency_value': 10}, 'id': 'call_Lt8qAvGpyNFPH1jQn5k1YoBv', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 124, 'output_tokens':

In [18]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': 'call_obxBud7gMEUPJGeSrcLJ0qov',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 10},
  'id': 'call_Lt8qAvGpyNFPH1jQn5k1YoBv',
  'type': 'tool_call'}]

In [31]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)



In [32]:
print(tool_call)


{'name': 'convert', 'args': {'base_currency_value': 10, 'conversion_rate': 91.8161}, 'id': 'call_Lt8qAvGpyNFPH1jQn5k1YoBv', 'type': 'tool_call'}


In [33]:
messages

[HumanMessage(content='What is the conversion factor between USD and INR, and based on that can you convert 10 usd to inr', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 124, 'total_tokens': 176, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-D2IeIYNjDngshaT34T0dNWGjDuyA5', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bfae2-9aa4-7be3-a5bf-f9a445c2332d-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'base_currency': 'USD', 'target_currency': 'INR'}, 'id': 'call_obxBud7gMEUPJGeSrcLJ0qov', 'type': 'tool_call'}, {'name': 'convert', 'args

In [ ]:
llm_with_tools.invoke(messages).content ### DEPRICATED METHOD

BadRequestError: Error code: 400 - {'error': {'message': "Invalid parameter: messages with role 'tool' must be a response to a preceeding message with 'tool_calls'.", 'type': 'invalid_request_error', 'param': 'messages.[4].role', 'code': None}}

In [ ]:
from langchain.agents import create_agent ## NEW METHOD 2026
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

# 1. Define tools with clear docstrings (the agent uses these to "reason")
@tool
def get_weather(location: str):
    """Returns the current weather for a given city."""
    return f"The weather in {location} is 19°C with Haze."

llm = ChatOpenAI(model="gpt-4o")
tools = [get_weather]

# 2. Initialize the modern Agent
agent = create_agent(
    llm, 
    tools=tools, 
    debug=True,
    system_prompt="You are a helpful weather assistant."
)

# 3. Invoke with the standard v1.x message format
response = agent.invoke({"messages": [("user", "What is the weather in Bhopal?")]})

# 4. Access the final message safely
print("\nFinal Answer:", response["messages"][-1].content)

[values] {'messages': [HumanMessage(content='What is the weather in Bhopal?', additional_kwargs={}, response_metadata={}, id='805c15be-cc63-496e-8544-773c9a7e3a59')]}
[updates] {'model': {'messages': [AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 62, 'total_tokens': 78, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a0e9480a2f', 'id': 'chatcmpl-D2J2YjD3XbsufqmXwfz9XU7psWNZ4', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bfaf9-9132-74d2-aecc-2a526341be44-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'Bhopal'}, 'id': 'call_jG1XRmKfqr0zYfHuGh9kYBbC', 'type': 'tool_call'}], invalid_tool_calls=[], 